In [59]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from metrics import *
from data_prepare import final_testloader
from models import GCNN, AttGNN, GCNN_mutual_attention, GCNN_desc_pool, GCNN_with_descriptors


device = torch.device("cuda:0") if torch.cuda.is_available() else torch.cuda("cpu")
# model = GCNN_mutual_attention(num_layers=1, dropout=0.3)
# model.load_state_dict(torch.load("../masif_features/GCN.pth"))

# model = GCNN()
# model.load_state_dict(torch.load("../masif_features/GCN_baseline.pth"))

# model = GCNN_desc_pool()
# model.load_state_dict(torch.load("../masif_features/GCN_pool.pth"))

model = GCNN_with_descriptors(num_layers=1, dropout=0.3, num_features_pro=1024, output_dim=128, descriptor_dim=80, transformer_dim=31, nhead=4, dim_feedforward=128)
model.load_state_dict(torch.load("../masif_features/GCN_03_double_transforer.pth"))

model.to(device)
model.eval();

GCNN Loaded


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [60]:
import os
import torch
import glob
import numpy as np 
import random
import math
from os import listdir
from os.path import isfile, join
from torch.utils.data import Dataset as Dataset_n
from torch_geometric.data import DataLoader as DataLoader_n
from torch_geometric.data import Data

# npy_file = "../masif_features/testset.npy"
# processed_dir="../masif_features/processed/"
# def bump(g):
#     return g
#     # return Data.from_dict(g.__dict__)

# class LabelledDataset(Dataset_n):
#     def __init__(self, npy_file, processed_dir):
#       self.npy_ar = np.load(npy_file)
#       self.processed_dir = processed_dir
#       self.protein_1 = self.npy_ar[:,2]
#       self.protein_2 = self.npy_ar[:,5]
#       self.label = self.npy_ar[:,6].astype(float)
#       self.n_samples = self.npy_ar.shape[0]

#     def __len__(self):
#       return(self.n_samples)

#     def __getitem__(self, index):
#       prot_1 = os.path.join(self.processed_dir, self.protein_1[index]+".pt")
#       prot_2 = os.path.join(self.processed_dir, self.protein_2[index]+".pt")
#       # print(prot_1, prot_2)
#       # print(glob.glob(prot_1), glob.glob(prot_2))
#       #print(f'Second prot is {prot_2}')
#       prot_1 = torch.load(glob.glob(prot_1)[0], weights_only=True)
#       #print(f'Here lies {glob.glob(prot_2)}')
#       prot_2 = torch.load(glob.glob(prot_2)[0], weights_only=True)
#       prot_1 = bump(prot_1)
#       prot_2 = bump(prot_2)
#       prot_1.x = prot_1.x.to(torch.float32)
#       prot_2.x = prot_2.x.to(torch.float32)
#       return prot_1, prot_2, torch.tensor(self.label[index])



# dataset = LabelledDataset(npy_file = npy_file ,processed_dir= processed_dir)

In [61]:
testloader = final_testloader # DataLoader_n(dataset=final_testset, batch_size=4, num_workers=0)

In [62]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from metrics import *
from models import GCNN, AttGNN, GCNN_mutual_attention
from tqdm.auto import tqdm


# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.cuda("cpu")
# model = GCNN_mutual_attention(dropout=0.3, num_layers=1)
# model.load_state_dict(torch.load("../masif_features/GCN.pth")) #path to load the model
# model.to(device)
# model.eval()
predictions = torch.Tensor()
labels = torch.Tensor()
with torch.no_grad():
    for prot_1, prot_2, label, mas1_straight, mas1_flipped, mas2_straight, mas2_flipped in tqdm(testloader):
      prot_1 = prot_1.to(device)
      prot_2 = prot_2.to(device)
      #print("H")
      #print(torch.Tensor.size(prot_1.x), torch.Tensor.size(prot_2.x))
      mas1_straight = mas1_straight.to(device)
      mas1_flipped = mas1_flipped.to(device)
      mas2_straight = mas2_straight.to(device)
      mas2_flipped = mas2_flipped.to(device)
      output = model(prot_1, prot_2, mas1_straight, mas1_flipped, mas2_straight, mas2_flipped)
      predictions = torch.cat((predictions, output.cpu()), 0)
      labels = torch.cat((labels, label.view(-1,1).cpu()), 0)
labels = labels.numpy().flatten()
predictions = predictions.numpy().flatten()

  0%|          | 0/1921 [00:00<?, ?it/s]

In [63]:
mse = get_mse(labels, predictions)
acc = get_accuracy(labels, predictions, 0.5)
prec = precision(labels, predictions, 0.5)
sensitivity = sensitivity(labels, predictions,  0.5)
specificity = specificity(labels, predictions, 0.5)
f1 = f_score(labels, predictions, 0.5)
mcc = mcc(labels, predictions,  0.5)
auroc = auroc(labels, predictions)
auprc = auprc(labels, predictions)


print(f'mse : {mse}')
print(f'Accuracy : {acc}')
print(f'precision: {prec}')
print(f'Sensititvity :{sensitivity}')
print(f'specificity : {specificity}')
print(f'f-score : {f1}')
print(f'MCC : {mcc}')
print(f'AUROC: {auroc}')
print(f'AUPRC: {auprc}')

mse : 36.4664192199707
Accuracy : 89.48190575370997
precision: 0.8843861740166865
Sensititvity :0.9199107364244979
specificity : 0.8670868731159221
f-score : 0.9017987360233348
MCC : 0.7893734218370622
AUROC: 0.9553775215801509
AUPRC: 0.9539667799833158


In [64]:
import random
import time
import traceback
from copy import deepcopy

import requests
import os

ID = '[2]'

def alarm_message(message: str) -> None:
    """Отправляем message в @alarm_bot"""
    alarm = 'https://alarmerbot.ru/'
    key = "b2b834-4b2bae-6ebc49"
    try:
        requests.post(alarm, data={'key': key, 'message': ID + ' ' + message})
    except Exception as ex:
        print('alarm error for {}: {}'.format(key, str(type(ex))))


# last_len = len(open('logging.txt', 'r').readlines())

# while True:
#     cur = open('logging.txt', 'r').readlines()
#     if len(cur) > last_len:
#         for line in cur[last_len:]:
#             alarm_message(line)
#         last_len = len(cur)


In [65]:
alarm_message('finished')